In [7]:
# INSPIRED BY: https://docs.microsoft.com/en-us/sql/advanced-analytics/tutorials/python-ski-rental-linear-regression?view=sql-server-ver15

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from revoscalepy import RxComputeContext, RxInSqlServer, RxSqlServerData
from revoscalepy import rx_import

conn_str = 'Driver=SQL Server;Server=DEVNOTEBOOK\MSSQLSERVER01;Database=ML;Trusted_Connection=True;'

column_info = {
    "ConditionCode": { "type": "integer" },
    "Sqft" : { "type" : "integer" },         
    "NumBaths" : { "type" : "integer" },
    "NumBedrooms" : { "type" : "integer" },
    "Age" : { "type" : "integer" },
    "TotalAV" : { "type" : "integer" },                  
}

# Get data in intermediate format (I think)
data_source = RxSqlServerData(
    table="dbo.AssessmentTrainingDataLinReg", 
    connection_string=conn_str, 
    column_info=column_info
)
# Set up compute context
RxInSqlServer(
    connection_string=conn_str, 
    num_tasks=1, 
    auto_cleanup=False
)
# convert data to pandas dataframe
df = pd.DataFrame(rx_import(input_data = data_source))

training_set = df.sample(
    frac=0.8, 
    random_state=1
)
# "Use anything not present in the training set for the testing set."
testing_set = df.loc[~df.index.isin(training_set.index)]

linear_regression_model = LinearRegression()

x_train = training_set["Sqft"].values.reshape(-1,1)
y_train = training_set["TotalAV"]

x_test = testing_set["Sqft"].values.reshape(-1,1)
y_test = testing_set["TotalAV"]

linear_regression_model.fit(x_train, y_train)

print(
    "Training data shape:", 
    training_set.shape
)
print(
    "Testing data shape:", 
    testing_set.shape
)

linear_regression_model.fit(x_train, y_train)

# Generate our predictions for the test set.
y_predictions = linear_regression_model.predict(x_test)

error = mean_squared_error(
    linear_regression_predictions, 
    y_test
)

print(
    "Error:", 
    error
)


Rows Read: 10411, Total Rows Processed: 10411, Total Chunk Time: 0.026 seconds 
Training data shape: (8329, 6)
Testing data shape: (2082, 6)
Error: 32017.1442643
